In [7]:
#Dependencies
import tweepy
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from config import consumer_key, consumer_secret, access_token, access_token_secret

#Start Vader Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [8]:
#API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [10]:
#Create list of targets
target_terms = ("@CNN", "@BBC", "@CBS", "@FoxNews", "@nytimes")

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

#Results holder
results_list = []

#Variable to keep track of number of tweets
counter = 0

#Loop through target terms
for target in target_terms:
    
    #Hold oldest tweet
    oldest_tweet = None
    
    #Sentiment Variables
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    
    #Loop through the 100 most recent tweets
    for x in range(5):
        
        #Run search through tweets
        public_tweets = api.search(target, count=100, result_type="recent", max_id=oldest_tweet)
        
        #Loop through all tweets
        for tweet in public_tweets["statuses"]:
            
            #Use filters to make sure tweets are from a real person
            if (tweet["user"]["followers_count"] < max_followers
                and tweet["user"]["statuses_count"] > min_tweets
                and tweet["user"]["statuses_count"] < max_tweets
                and tweet["user"]["friends_count"] < max_following
                and tweet["user"]["lang"] == lang):
        
                #Run vader analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]
                date = tweet["created_at"]
        
                #Add the values to our earlier lists
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
        
            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1
        
        # Store the Average Sentiments
        sentiment = {
            "News Source": target,
            "Timestamp": date,
            "Compound": np.mean(compound_list),
            "Positive": np.mean(positive_list),
            "Neutral": np.mean(negative_list),
            "Negative": np.mean(neutral_list),
            "Tweet Count": len(compound_list)
        }
        
        
#         #Print the sentiments
#         print(sentiment)
#         print()
    
        #Add to the results_list
        results_list.append(sentiment)

print("Complete!")

TweepError: [{'code': 32, 'message': 'Could not authenticate you.'}]

In [14]:
#Create a datafram
news_df = pd.DataFrame(results_list).set_index("News Source")
news_df.head()

,Compound,Negative,Neutral,Positive,Timestamp,Tweet Count
News Source,,,,,,
@CNN,-0.008998,0.874000,0.068038,0.058000,Tue Jun 26 02:54:51 +0000 2018,53
@CNN,-0.012476,0.858318,0.074542,0.067187,Tue Jun 26 02:54:08 +0000 2018,107
@CNN,-0.007062,0.847662,0.076149,0.076221,Tue Jun 26 02:53:19 +0000 2018,154
@CNN,-0.022506,0.841755,0.082195,0.076080,Tue Jun 26 02:52:25 +0000 2018,200
@CNN,-0.028301,0.853837,0.077829,0.068354,Tue Jun 26 02:51:29 +0000 2018,246
